In [14]:
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
...,...,...,...,...,...
130,M9Y\n,Not assigned\n,M1Z\n,43.648690,-79.385440
131,M2Z\n,Not assigned\n,M3Z\n,43.648690,-79.385440
132,M4Z\n,Not assigned\n,M5Z\n,43.648690,-79.385440
133,M6Z\n,Not assigned\n,M7Z\n,43.648690,-79.385440


In [8]:
neighbors.drop(neighbors[neighbors['Borough']=="Not assigned\n"].index,axis=0, inplace=True)

In [16]:
Toronto2 = neighbors[~np.isnan(neighbors.Latitude)]
Toronto2.reset_index(drop=True)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
1,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390
...,...,...,...,...,...
86,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.653340,-79.509766
87,M4Y\n,Downtown Toronto\n,Church and Wellesley\n,43.666659,-79.381472
88,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
89,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


In [17]:
Toronto2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
5,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390


In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
CLIENT_ID = 'PB0VXS1VKOUZHYM4C0ODHSL4YKCINOHH0ZJ51LUSXZMMXENI' # your Foursquare ID
CLIENT_SECRET = '5NEWWZ3WZXQGGKABUVT2SCUJ4ZACI3CGQE3KOLPBQUMZKXTY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PB0VXS1VKOUZHYM4C0ODHSL4YKCINOHH0ZJ51LUSXZMMXENI
CLIENT_SECRET:5NEWWZ3WZXQGGKABUVT2SCUJ4ZACI3CGQE3KOLPBQUMZKXTY


In [24]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
              CLIENT_ID, 
              CLIENT_SECRET, 
              VERSION, 
              lat, 
              lng, 
              radius, 
              LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(Toronto2['Latitude'], Toronto2['Longitude'], Toronto2['Borough'], Toronto2['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto